In [46]:
import pandas as pd
import os
import numpy as np
from ensure import ensure_annotations
import re
import warnings
warnings.filterwarnings('ignore')

In [47]:
print(os.getcwd())
print(os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem'))
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [48]:
df = pd.read_csv(r'data\raw_data\apple_mobile_new_data.csv',index_col=None)

In [49]:
df.drop_duplicates(inplace=True,ignore_index=True)

## Battery

In [50]:
df = df.drop(labels=['battery'],axis=1)

## Storage 

In [51]:
def extract_storage(row):
    storage = row.split()[1]
    return storage
df['storage'] = df['storage_ram'].apply(func=extract_storage)

In [52]:
import warnings
warnings.filterwarnings('ignore')
count = 0
for i in range(0,len(df['storage'])):
    df['storage'][i]= df['storage'].iloc[i].replace('Storage','')
    count+=1
print(f'looped till {count} index')

looped till 424 index


In [53]:
df['storage'] = df['storage'].replace('1','1024')

In [54]:
df['storage'] = df['storage'].astype('Int16')

## Processors


In [55]:
def extract_processor(text):
    pattern = r'.*Processor'
    match = re.findall(pattern, text)
    if match:
        return match[0]
    else:
        return text


In [56]:
df['processors'] = df['os_processor'].apply(func=extract_processor)

## IOS Version

In [57]:
def ios_version(text_):
    pattern=r'^\d+[Processor]|\d+[.Processor]'
    found = re.findall(pattern,text_)
    if found:
        return found[0]
    else:
        return text_

In [58]:
df['ios_version'] = df['processors'].apply(ios_version)

In [59]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['ios_version']):
    # print(i)
    i = str(i).replace('P','')
    i = str(i).replace('.','')
    df['ios_version'][Index] = i

In [60]:
non_numeric = [i for i in df['ios_version'] if len(i)> 2]
non_numeric

['Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100',
 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100']

In [61]:
df[df['ios_version'] == 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100'] # Just Droping

,index,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,storage,processors,ios_version
317,317,"APPLE IPhone 4 (Black, 8 GB)",4.5,"₹22,900",https://rukminim2.flixcart.com/image/312/312/m...,Internal Storage8 GBPhone Book MemoryYesCall L...,Operating SystemiOS 7Operating FrequencyGSM - ...,Primary Camera AvailableYesPrimary Camera5MP R...,Display Size8.89 cm (3.5 inch)Resolution640 x ...,Network Type3GSupported NetworksGSMInternet Co...,8,Operating SystemiOS 7Operating FrequencyGSM - ...,Operating SystemiOS 7Operating FrequencyGSM - ...
324,324,"APPLE IPhone 4 (White, 8 GB)",4.6,"₹13,999",https://rukminim2.flixcart.com/image/312/312/m...,Internal Storage8 GBPhone Book MemoryYesCall L...,Operating SystemiOS 7Operating FrequencyGSM - ...,Primary Camera AvailableYesPrimary Camera5MP R...,Display Size8.89 cm (3.5 inch)Resolution640 x ...,Network Type3GSupported NetworksGSMInternet Co...,8,Operating SystemiOS 7Operating FrequencyGSM - ...,Operating SystemiOS 7Operating FrequencyGSM - ...


In [62]:
df = df.drop(df[df['ios_version'] == 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100'].index)

In [63]:
df['ios_version'] = df['ios_version'].astype('Int16')